In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
cd '../gdrive/MyDrive/SSAC/3조'

/gdrive/.shortcut-targets-by-id/15_BxZVEQYCdGCGiQ5nexpWPc1cgHVe4w/3조


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 376 kB/s 
     |████████████████████████████████| 448 kB 59.2 MB/s 


In [ ]:
from konlpy.tag import Kkma, Okt
from konlpy.utils import pprint

# 텍스트 전처리 📰

In [ ]:
df = pd.read_csv('data/content.csv')
text_df = pd.read_csv('text_example.csv')
train_data = pd.read_csv('data/train.csv')

In [ ]:
import json 

with open('data/brand_dict.json', 'r') as f:
    brand_dict = json.load( f)

with open('data/cat_3_dict.json', 'r') as f:
    cat_3_dict = json.load(f)

## 필요 함수 정의

### 정규표현식 - 한글 제외 모두 제거

In [ ]:
import re 

def re_kor(s, target = 'list'):
    # pre = re.compile('[^ ㄱ-ㅣ가-힣]')
    pre = re.compile('[^ 가-힣]')
    result = pre.sub(' ', s) 
    while '  ' in result:
        result = result.replace('  ', ' ')
    if target == 'list':
        result = result.split(' ')
        result = [r for r in result if r!='']
    return result

### 단어 개수 세기

In [ ]:
def get_voca_cnt(voca_list):
    total_vocabs = []
    for t in voca_list:
        total_vocabs.extend(t)
    total_unique, total_count = np.unique(total_vocabs, return_counts=True)
    vocab_cnt = pd.DataFrame({'name': total_unique, 'cnt': total_count})
    return vocab_cnt.sort_values('cnt', ascending=False)

## Text_df(예시) 전처리

### 제목 : 정규표현식으로 한글만 남기기

In [ ]:
name_list = df.content_name.apply(re_kor)
name_list

0             [개인용달, 운송, 상담, 원룸이사]
1             [개인용달, 운송, 상담, 원룸이사]
2             [개인용달, 운송, 상담, 원룸이사]
3             [개인용달, 운송, 상담, 원룸이사]
4             [개인용달, 운송, 상담, 원룸이사]
                    ...           
575955    [빈티지, 향수병, 퍼퓸, 보틀, 스프레이]
575956            [빈티지, 플라워액자, 거울]
575957      [빈티지, 빅토리안, 앤틱박스, 보석함]
575958                    [폴로, 패딩]
575959          [폴로, 랄프로렌, 베어, 세트]
Name: content_name, Length: 575960, dtype: object

In [ ]:
def brand_(name, brand_dict):
    for key in brand_dict.keys():
        for value in brand_dict[key]:
            if value in name:
                name = name.replace(value, key)
    return name

### 제목 단어모음 만들기

In [ ]:
name_cnt = get_voca_cnt(name_list)
name_cnt

,name,cnt
108023,새상품,30137
228904,팝니다,17275
228363,판매합니다,16123
228235,판매,15988
186119,정품,14323
...,...,...
105284,사진찍고,1
105285,사진찍어주실분,1
105287,사진참고부탁드려요,1
105289,사진참고해주세요,1


### 키워드 : "," 기준으로 구분해주기

In [ ]:
keyword_list = df.keyword.fillna('').apply(lambda x: x.split(','))
keyword_list

0                                            []
1                                            []
2                                            []
3                                            []
4                                            []
                          ...                  
575955                 [빈티지퍼퓸보틀, 빈티지향수병, 빈티지퍼퓸]
575956                                  [빈티지액자]
575957             [빈티지보석함, 앤틱박스, 앤틱보석함, 빈티지박스]
575958                                   [폴로패딩]
575959    [폴로베어, 곰인형, 폴로악세사리, 신라스테이곰인형, 노티드곰인형]
Name: keyword, Length: 575960, dtype: object

### 제목 단어모음 만들기

In [ ]:
keyword_cnt = get_voca_cnt(keyword_list)
keyword_cnt

,name,cnt
0,,94495
75622,나이키,8127
337554,파름신오신날,6253
50179,강아지옷,5131
321539,크로스백,4211
...,...,...
147592,바디랩스,1
147591,바디랩,1
147588,바디드라이기,1
147587,바디닥터스프로폴리,1


## 제목-키워드 단어 모음 만들기

In [ ]:
total_vocab = pd.merge(name_cnt, keyword_cnt,how='outer', on='name').fillna(0)
total_vocab['cnt'] = (total_vocab['cnt_x'] + total_vocab['cnt_y']).astype(int)
total_vocab = total_vocab[['name', 'cnt']]
total_vocab

,name,cnt
0,새상품,32845
1,팝니다,17346
2,판매합니다,16141
3,판매,16415
4,정품,16751
...,...,...
524738,바디럽매트리스,1
524739,바디럽마약베개,1
524740,바디드라이기,1
524741,바디닥터스프로폴리,1


### 3번 이상 언급된 단어만 ?

In [ ]:
# total_vocab= total_vocab[total_vocab.cnt>2]

### 카테고리

In [ ]:
cat_list1 = df.content_cat_1.apply(lambda x: [cat.replace('기타', 'etc').replace(' ', '') for cat in x.split('/')])
cat_list2 = df.content_cat_2.apply(lambda x: x.replace('/', ' ')).apply(lambda x: cat_3_dict[x] if x in cat_3_dict.keys() else x).apply(lambda x: x.split(' '))
cat_list3 = df.content_cat_3.apply(lambda x: x.replace('/', ' ')).apply(lambda x: cat_3_dict[x] if x in cat_3_dict.keys() else x).apply(lambda x: x.split(' '))
cat_list = cat_list1 + cat_list2 + cat_list3

## Description : Kkma, re 📑

In [ ]:
kkma = Kkma()

In [ ]:
# kkma로 명사 추출 후 단어 모음에 있는 단어만 남기기
def kkma_voca(sentence):
    voca_list = kkma.nouns(sentence)
    return [voca for voca in voca_list if voca in total_vocab.name.tolist()]

# 정규표현식으로 전처리 후 단어 모음에 있는 단어만 남기기
def re_voca(sentence):
    voca_list = re_kor(sentence)
    return [voca for voca in voca_list if voca in total_vocab.name.tolist()]

# kkma, 정규표현식으로 나온 각 리스트 합쳐서 한번에
def re_kkma_voca(sentence):
    voca_list1 = kkma.nouns(sentence)
    voca_list2 = re_kor(sentence)
    voca_list = voca_list1 + voca_list2
    return [voca for voca in voca_list if voca in total_vocab.name.tolist()]

- 상세설명 결측 채우기

In [ ]:
text_df.description = text_df.description.fillna('!')

- 전처리 진행

In [ ]:
# kkma 전처리
desc_list = text_df.description.apply(kkma_voca)
# # kkma + re 전처리
# desc_list = text_df.description.apply(re_kkma_voca)

# Word2Vec 학습

### 각 컨텐츠마다 한 문장 만들기(이름 + 키워드 + 상세설명 + 카테고리)

In [ ]:
total_sentences = name_list + keyword_list + cat_list
total_sentences

0             [개인용달, 운송, 상담, 원룸이사, , 지역서비스, 이사, 용달, 이사, 용달]
1             [개인용달, 운송, 상담, 원룸이사, , 지역서비스, 이사, 용달, 이사, 용달]
2             [개인용달, 운송, 상담, 원룸이사, , 지역서비스, 이사, 용달, 이사, 용달]
3             [개인용달, 운송, 상담, 원룸이사, , 지역서비스, 이사, 용달, 이사, 용달]
4             [개인용달, 운송, 상담, 원룸이사, , 지역서비스, 이사, 용달, 이사, 용달]
                                ...                        
575955    [빈티지, 향수병, 퍼퓸, 보틀, 스프레이, 빈티지퍼퓸보틀, 빈티지향수병, 빈티지퍼...
575956               [빈티지, 플라워액자, 거울, 빈티지액자, etc, etc, etc]
575957    [빈티지, 빅토리안, 앤틱박스, 보석함, 빈티지보석함, 앤틱박스, 앤틱보석함, 빈티...
575958                        [폴로, 패딩, 폴로패딩, etc, etc, etc]
575959    [폴로, 랄프로렌, 베어, 세트, 폴로베어, 곰인형, 폴로악세사리, 신라스테이곰인형...
Length: 575960, dtype: object

## Word2Vec 학습

In [ ]:
from gensim.models import Word2Vec

model = Word2Vec(sentences = total_sentences, size = 5, window = 5, min_count = 1, workers = 4, sg = 1)

## 유사도 확인

In [ ]:
print(name_list[0])
print(keyword_list[0])
model.wv.n_similarity(name_list[0], keyword_list[0])

['스튜디오', '톰보이', '패딩']
['스튜디오톰보이', '노스페이스', '뉴발란스', '숏패딩', '패딩']


0.88625646

In [ ]:
i, j = 130, -8
print(name_list.iloc[i])
print(keyword_list.iloc[j])
model.wv.n_similarity(name_list.iloc[i], keyword_list.iloc[j])

['맥케이지', '여성패딩', '사이즈']
['꼼데가르송', '꼼데가르송가디건', '꼼데가르송반팔', '꼼데가르송컨버스', '꼼데가르송셔츠']


0.83096015

In [ ]:
for i in range(50):
    print(name_list.iloc[8104])
    print(keyword_list.iloc[i])
    print(desc_list.iloc[i])
    print('제목-키워드', model.wv.n_similarity(name_list.iloc[8104], keyword_list.iloc[i]))
    try:
        print('제목-설명', model.wv.n_similarity(name_list.iloc[8104], desc_list.iloc[i]))
    except:
        pass
    print('----------------------------------------------------------------------------')


## FastText

새로운 단어에 대응 가능

In [ ]:
from gensim.models import FastText
model_ft = FastText(size=100, window=5, min_count=3)
model_ft.build_vocab(sentences=total_sentences)
model_ft.train(sentences=total_sentences, total_examples=len(total_sentences), epochs=10)

In [ ]:
model_ft.wv.similarity('나이키', '아디다스')

0.41764084

In [ ]:
model_ft.wv.n_similarity(['아디다스', '패딩'], ['꼼데가르송', '가디건'])

0.27248108

In [ ]:
model_ft.wv.n_similarity(['아디다스패딩나이키'], ['나이키', '아디다스', '패딩'])

0.80131626

In [ ]:
model_ft.wv.wmdistance('아디다스패딩나이키', '나이키 아디다스 패딩')

0.8193344027309417

In [ ]:
from gensim.similarities import WmdSimilarity

In [ ]:
wmd = WmdSimilarity('아디다스 패딩 나이키', w2v_model=model_ft)

In [ ]:
wmd.get_similarities('나이키패딩아디다스')

array([0.51215218, 0.48941444, 0.49421617, 0.52362626, 0.43490708,
       0.        , 0.        , 0.43490708, 0.50917311, 0.51549167,
       0.46689056])